In [136]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix, recall_score, accuracy_score, precision_score, f1_score
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

loaded in cleaned dataframe from Exploratory Data Analysis in Jupyter Notebook folder in this repository

In [112]:
df = pd.read_csv('../data/clean_df.csv', index_col=0) # loaded up data and had two index colums so set to only have 1

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   int64  
 3   international plan      3333 non-null   int64  
 4   voice mail plan         3333 non-null   int64  
 5   number vmail messages   3333 non-null   int64  
 6   total day minutes       3333 non-null   float64
 7   total day calls         3333 non-null   int64  
 8   total day charge        3333 non-null   float64
 9   total eve minutes       3333 non-null   float64
 10  total eve calls         3333 non-null   int64  
 11  total eve charge        3333 non-null   float64
 12  total night minutes     3333 non-null   float64
 13  total night calls       3333 non-null   int64  
 14  total night charge      3333 non-null   

In [114]:
X = df.drop(columns='churn')
y = df.churn

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

.

In [115]:
dr = DummyClassifier()

dummy_model = dr.fit(X_train, y_train)
dummy_train =dr.score(X_train,y_train)
dummy_test = dr.score(X_test,y_test)

print(f'Train: {dummy_train}')
print(f'Test : {dummy_test}')

Train: 0.8527410964385754
Test : 0.8621103117505995


In [116]:
cat_cols = df[['state','area code']]

num_cols = df.drop(columns=['state','area code', 'churn', 'international plan', 'voice mail plan'])

#binary_cols = df[['international plan', 'voice mail plan']]

In [117]:
subpipe_num = Pipeline(steps=[('ss', StandardScaler())])
subpipe_cat = Pipeline(steps=[('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [118]:
num_cols.columns

Index(['account length', 'number vmail messages', 'total day minutes',
       'total day calls', 'total day charge', 'total eve minutes',
       'total eve calls', 'total eve charge', 'total night minutes',
       'total night calls', 'total night charge', 'total intl minutes',
       'total intl calls', 'total intl charge', 'customer service calls'],
      dtype='object')

In [119]:
ct = ColumnTransformer(transformers= [
    ('subpipe_num', subpipe_num, num_cols.columns),
    ('subpipe_cat', subpipe_cat, [0,2])
], remainder='passthrough')

In [120]:
pd.DataFrame(ct.fit_transform(X_train))

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
0,0.527291,-0.582315,-0.417150,1.682410,-0.416881,-0.303407,1.120060,-0.302652,-0.083592,-2.121425,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-1.376394,-0.582315,-1.055002,-1.691942,-1.055485,-0.648624,-1.906901,-0.649130,-0.130865,-0.108357,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.023776,1.547862,0.615477,1.329865,0.615205,-0.478941,-0.897914,-0.479333,0.113378,-0.005122,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.151564,-0.582315,1.132704,0.070779,1.132324,-1.434626,0.867814,-1.433868,-0.312077,-0.418060,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.429037,-0.582315,0.403468,1.279502,0.403413,-0.720788,-0.696116,-0.720261,-0.119047,-0.830997,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2494,-0.349406,-0.582315,0.790932,0.423323,0.790445,-1.422923,-0.696116,-1.422395,1.730500,-0.882614,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2495,-0.574842,-0.582315,0.240807,-0.785400,0.241074,-0.553055,0.262421,-0.552759,-1.415108,0.975603,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2496,-1.025715,-0.582315,0.255428,0.876594,0.255050,-2.251833,0.514668,-2.250729,0.170499,1.749860,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2497,-0.073873,-0.582315,-1.957865,0.322596,-1.957485,-0.274152,0.161523,-0.275118,-0.514956,0.511049,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [121]:
ct_no_scaler = ct = ColumnTransformer(transformers= [
    ('subpipe_cat', subpipe_cat, [0,2])
], remainder='passthrough')

In [122]:
pd.DataFrame(ct_no_scaler.fit_transform(X_train))

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,184.9,122.0,15.72,197.2,59.0,8.87,8.5,5.0,2.30,4.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,167.2,62.0,14.21,194.8,98.0,8.77,9.7,6.0,2.62,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,175.9,82.0,14.95,207.2,100.0,9.32,8.9,3.0,2.40,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,126.9,117.0,10.79,185.6,92.0,8.35,11.7,6.0,3.16,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,163.5,86.0,13.90,195.4,84.0,8.79,10.4,6.0,2.81,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,127.5,86.0,10.84,289.3,83.0,13.02,14.5,4.0,3.92,3.0
2495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,172.1,105.0,14.63,129.6,119.0,5.83,10.2,1.0,2.75,0.0
2496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,85.0,110.0,7.23,210.1,134.0,9.45,13.2,8.0,3.56,3.0
2497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,186.4,103.0,15.84,175.3,110.0,7.89,10.5,4.0,2.84,3.0


In [123]:
imb_pipe_logreg = ImPipeline(steps=[('ct',ct_no_scaler),
                            ('sm', SMOTE(random_state=3, sampling_strategy='minority')),
                            ('logreg',LogisticRegression(random_state=3, max_iter=5000))])

In [124]:
imb_pipe_logreg.fit(X_train, y_train)
cross_val_score(imb_pipe_logreg, X_train, y_train).mean()

0.775112625250501

In [138]:
imb_pipe_knn = ImPipeline(steps=[('ct',ct),
                            ('sm', SMOTE(random_state=3, sampling_strategy='minority')),
                            ('knn',KNeighborsClassifier())])

In [139]:
imb_pipe_knn.fit(X_train, y_train)
cross_val_score(imb_pipe_knn, X_train, y_train).mean()

0.6694693386773547

In [140]:
imb_pipe_dtc = ImPipeline(steps=[('ct',ct),
                            ('sm', SMOTE(random_state=3, sampling_strategy='minority')),
                            ('dtc',DecisionTreeClassifier(random_state=3))])

In [141]:
imb_pipe_dtc.fit(X_train, y_train)
cross_val_score(imb_pipe_rfc, X_train, y_train).mean()

0.9295735470941885

In [142]:
imb_pipe_rfc = ImPipeline(steps=[('ct',ct),
                            ('sm', SMOTE(random_state=3, sampling_strategy='minority')),
                            ('rfc',RandomForestClassifier(random_state=3))])

In [143]:
imb_pipe_rfc.fit(X_train, y_train)
cross_val_score(imb_pipe_rfc, X_train, y_train).mean()

0.9295735470941885